In [ ]:
# This is a CNN, where the model is trained to classify images to determine whether it is a dog or a cat.
# PROBLEM: This model is currently suffering from overfitting. The accuracy on the training dataset reaches
# 100, while the validation data accuracy is around 67%. 

In [ ]:
# Importing all necessary libraries to prepare image data, train CNN, and test the nueral network
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
import itertools
import os
import shutil
import random
import glob
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
%matplotlib inline

In [ ]:
# Checking if a GPU is being used
physical_devices = tf.config.experimental.list_physical_devices('GPU')
print("Num GPUs Available: ", len(physical_devices))

In [ ]:
# Organizes the data into training, validating, and testing directories. Each of these have a separate directly for
# cats and dogs.
os.chdir('/Users/sivamagarwalla/Downloads/dogs-vs-cats')
if os.path.isdir('train/dog') is False:
    os.makedirs('train/dog')
    os.makedirs('train/cat')
    os.makedirs('valid/dog')
    os.makedirs('valid/cat')
    os.makedirs('test/dog')
    os.makedirs('test/cat')
    
    for i in random.sample(glob.glob('cat*'), 1000):
        shutil.move(i, 'train/cat')      
    for i in random.sample(glob.glob('dog*'), 1000):
        shutil.move(i, 'train/dog')
    for i in random.sample(glob.glob('cat*'), 200):
        shutil.move(i, 'valid/cat')        
    for i in random.sample(glob.glob('dog*'), 200):
        shutil.move(i, 'valid/dog')
    for i in random.sample(glob.glob('cat*'), 100):
        shutil.move(i, 'test/cat')      
    for i in random.sample(glob.glob('dog*'), 100):
        shutil.move(i, 'test/dog')

In [ ]:
train_path = '/Users/sivamagarwalla/Downloads/dogs-vs-cats/train'
valid_path = '/Users/sivamagarwalla/Downloads/dogs-vs-cats/valid'
test_path = '/Users/sivamagarwalla/Downloads/dogs-vs-cats/test'

In [ ]:
# Using Keras' ImageDataGenerator class to create batches of data
train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=train_path, target_size=(224,224), classes=['cat', 'dog'], batch_size=10, shuffle=True)
valid_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=valid_path, target_size=(224,224), classes=['cat', 'dog'], batch_size=10, shuffle=True)
test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=test_path, target_size=(224,224), classes=['cat', 'dog'], batch_size=10, shuffle=False)

In [ ]:
# Generating a batch of images and labels from the training data to allow us to visualize the data.
imgs, labels = next(train_batches)

# Plotting function obtained from TensorFlow's documentation to plot the processed images
def plotImages(images_arr):
    fig, axes = plt.subplots(1, 10, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip( images_arr, axes):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()
    
plotImages(imgs)
print(labels)

In [ ]:
model = Sequential([
    Conv2D(filters=32, kernel_size=(3,3), activation="relu", padding="same", input_shape=(224, 224, 3)),
    MaxPool2D(pool_size=(2, 2), strides=2),
    Conv2D(filters=64, kernel_size=(3,3), activation="relu", padding="same"),
    MaxPool2D(pool_size=(2, 2), strides=2),
    Flatten(),
    Dense(units=2, activation="softmax"),
])

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss="categorical_crossentropy", metrics=["accuracy"])

In [ ]:
model.fit(x=train_batches, validation_data=valid_batches, epochs=10, verbose=2)

In [ ]:
test_imgs, test_labels = next(test_batches)
plotImages(test_imgs)
print(test_labels)

In [ ]:
test_batches.classes

In [ ]:
predictions = model.predict(x=test_batches, verbose=0)
np.round(predictions)

In [ ]:
cm = confusion_matrix(y_true=test_batches.classes, y_pred=np.argmax(predictions, axis=-1))

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
test_batches.class_indices

cm_plot_labels = ['cat','dog']
plot_confusion_matrix(cm=cm, classes=cm_plot_labels, title='Confusion Matrix')

In [ ]:
# Instead of focusing on overfitting for now, we will use the pre-trained VVG16 model and fine-tune it for the data that we are using.
# It is important to remember the input form must match the input form that was used to originally train this model.

In [ ]:
vgg16_model = tf.keras.applications.vgg16.VGG16()
vgg16_model.summary()

In [ ]:
# Creating a new sequential model and adding all the layers for VGG16 besides the last one. We want to classify between 2 classes, not 1000.
model = Sequential()
for layer in vgg16_model.layers[:-1]:
    model.add(layer)
model.summary()

In [ ]:
# Setting the layers trainability to false because we want to freeze these weights and biases. This
# is because the model has already been trained on dogs & cats and learned how to identify.
for layer in model.layers:
    layer.trainable = False
    
model.add(Dense(units=2, activation="softmax"))
model.summary()

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss="categorical_crossentropy", metrics=["accuracy"])
model.fit(x=train_batches, validation_data=valid_batches, epochs=5, verbose=2)

In [ ]:
predictions = model.predict(x=test_batches, verbose=0)
np.round(predictions)

test_batches.classes
cm = confusion_matrix(y_true=test_batches.classes, y_pred=np.argmax(predictions, axis=-1))
cm_plot_labels = ['cat','dog']
plot_confusion_matrix(cm=cm, classes=cm_plot_labels, title='Confusion Matrix')